In [ ]:
!pip uninstall interpret interpret-community -y
!pip install transformers lime datasets plotly


## LIME-based explanations for transformer text models
We treat the Hugging Face pipeline as a black-box classifier and use LIME to perturb the input text to estimate which tokens push the model toward each label.


In [ ]:
from transformers import pipeline
from lime.lime_text import LimeTextExplainer
import numpy as np
np.random.seed(42)

# Use a pretrained sentiment analysis model as an example black-box classifier
classifier = pipeline('text-classification', model='distilbert-base-uncased-finetuned-sst-2-english', return_all_scores=True)

# Identify the class labels that the pipeline returns
_sample_scores = classifier('This placeholder call collects label names.', truncation=True)[0]
class_names = [entry['label'] for entry in _sample_scores]
explainer = LimeTextExplainer(class_names=class_names)


In [ ]:
from IPython.display import display, HTML
from pathlib import Path
import matplotlib.pyplot as plt

def predict_proba(texts):
    """Wrap the Hugging Face pipeline so LIME can query class probabilities."""
    outputs = classifier(texts, truncation=True)
    return np.array([[entry['score'] for entry in sample] for sample in outputs], dtype=float)

def explain_text(text, num_features=10, show_html=False, show_figure=True, save_html_path=None):
    """Run LIME on a single text example, print weights, and render static and/or interactive views."""
    explanation = explainer.explain_instance(text, predict_proba, num_features=num_features)
    probabilities = predict_proba([text])[0]
    top_label = class_names[int(np.argmax(probabilities))]
    print(f'Model prediction: {top_label} (confidence={probabilities.max():.3f})')
    print('\nToken contributions (positive pushes prediction up, negative down):')
    for token, weight in explanation.as_list():
        print(f'{token}: {weight:+.3f}')

    if show_figure:
        fig = explanation.as_pyplot_figure()
        fig.tight_layout()
        plt.show()
    else:
        fig = None

    if show_html or save_html_path:
        html = explanation.as_html()
        if show_html:
            display(HTML(html))
        if save_html_path:
            Path(save_html_path).write_text(html, encoding='utf-8')
            print(f'Saved interactive HTML to {Path(save_html_path).resolve()}')

    return explanation, fig


In [ ]:
sample_text = 'The movie had gorgeous visuals but the storyline fell flat and predictable.'
lime_explanation, lime_fig = explain_text(
    sample_text,
    num_features=10,
    show_html=False,
    show_figure=True,
    save_html_path='lime_explanation.html'
)
